## 普通大模型

In [2]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

#加载环境
load_dotenv()

 # 初始化 LLM (确保使用较新的模型以支持良好的 Tool Calling)
llm = ChatOpenAI(
    model="deepseek-chat",
    temperature=0.5,
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url=os.getenv("DASHSCOPE_BASE_URL")
)


ai_msg = llm.invoke("写一首现代诗,并且写入到poem.txt")
print("模型输出:",ai_msg.content)

# 定义文件操作的安全根目录(防止AI随意修改系统文件)
WORK_DIR = "D:\\Agent资料"
if not os.path.exists(WORK_DIR):
    os.makedirs(WORK_DIR)

#迭代模型响应元数据
for key, value in ai_msg.response_metadata.items():
    print(f"{key}: {value}")

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

## Agent:接入工具的大模型

In [ ]:
import os
import shutil
from typing import Optional
from dotenv import load_dotenv

# 核心组件导入
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain.agents import create_agent

#加载环境
load_dotenv()

 # 初始化 LLM (确保使用较新的模型以支持良好的 Tool Calling)
llm = ChatOpenAI(
    model="deepseek-chat",
    temperature=0.5,
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url=os.getenv("DASHSCOPE_BASE_URL")
)

# 定义文件操作的安全根目录(防止AI随意修改系统文件)
WORK_DIR = "D:\\Agent资料"
if not os.path.exists(WORK_DIR):
    os.makedirs(WORK_DIR)



## 定义工具:查看磁盘，读写文件

In [ ]:
# 定义工具 Tools
@tool
def check_disk_usage() -> str:
    """
        查看磁盘使用情况。返回总空间、已用空间和剩余空间。
    """
    total, used, free = shutil.disk_usage("/")
    
    # 将字节转换为 GB
    gb = 2**30
    info = (
        f"磁盘状态:\n"
        f"- 总空间: {total / gb:.2f} GB\n"
        f"- 已用: {used / gb:.2f} GB\n"
        f"- 剩余: {free / gb:.2f} GB"
    )
    return info

@tool
def file_manager(action: str, filename: str, content: Optional[str] = None) -> str:
    """
    对指定路径的文件进行读写操作。
    
    Args:
        action (str): 操作类型，必须是 'read' (读取) 或 'write' (写入)。
        filename (str): 文件名（例如 'notes.txt'）。
        content (str, optional): 当 action 为 'write' 时，需要写入的内容。
    """
    filepath = os.path.join(WORK_DIR, filename)
    
    try:
        if action == "write":
            if content is None:
                return "错误: 写入模式需要提供 content 内容。"
            with open(filepath, "w", encoding="utf-8") as f:
                f.write(content)
            return f"成功: 已将内容写入 {filename}。"
            
        elif action == "read":
            if not os.path.exists(filepath):
                return f"错误: 文件 {filename} 不存在。"
            with open(filepath, "r", encoding="utf-8") as f:
                return f.read()
                
        else:
            return "错误: 不支持的操作。请使用 'read' 或 'write'。"
            
    except Exception as e:
        return f"发生异常: {str(e)}"

## 大模型执行工具，对外界进行交互操作

In [ ]:

# 工具列表
tools = [check_disk_usage, file_manager]

    
# 创建Agent
agent_executor = create_agent(llm, tools)

print("------ Agent 已启动 ------")

# 用户输入
query = "写一首关于友情的现代诗,写入到poem.txt"
    
print(f"\n用户: {query}")
print("Agent正在思考...\n")

# stream 流式输出
events = agent_executor.stream(
    {"messages": [HumanMessage(content=query)]},
    stream_mode="values"
)

for event in events:
    # event["messages"] 包含了当前的对话历史
    last_message = event["messages"][-1]
        
    # 打印 AI 的回复
    if last_message.type == "ai":
        # 如果有 tool_calls，说明正在调用工具
        if hasattr(last_message, "tool_calls") and len(last_message.tool_calls) > 0:
            print(f"Executing Tool: {last_message.tool_calls[0]['name']}")
        else:
            print(f"Final Answer: {last_message.content}")
        
    # 打印 Tool 的执行结果
    elif last_message.type == "tool":
        print(f"Tool Output: {last_message.content}")